當連到一個WebServer時，提供一個WebAPI，第一個動作是：透過HTTP requests先連線上去，可是有可能網路會中斷，所以檢查status_code=200 (代表沒問題)
確保應用程式的穩定性

- class datetime.datetime(year, month, day, hour=0, minute=0, second=0, microsecond=0, tzinfo=None, *, fold=0)

In [2]:
import requests
from requests import Response
from pydantic import RootModel,BaseModel,Field,field_validator
from datetime import datetime
import pandas as pd

class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')
    時間:datetime = Field(alias='mday')

    @field_validator('可借數量','可還數量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value

class Youbike(RootModel):
    root:list[Site]

try:
    youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
    youbike_url.raise_for_status()
except Exception as e:   #不管是什麼錯誤
    print(e)
else:
    data = Youbike.model_validate_json(youbike_url.text)
    all_sites = data.model_dump()
# model_dump 將對象轉化為Dict，之後就可以調用python標準庫序列化json字串，会序列化嵌套对象
# 也可以使用dict(model)将对象转化为字典，但嵌套对象不会被转化为字典。
df = pd.DataFrame(all_sites)
df

,行政區域,總數量,可借數量,可還數量,時間
0,大安區,28,5,23,2024-07-02 20:53:19
1,大安區,21,4,17,2024-07-02 20:53:20
2,大安區,16,10,6,2024-07-02 20:51:20
3,大安區,11,5,6,2024-07-02 20:51:20
4,大安區,16,4,12,2024-07-02 20:46:20
...,...,...,...,...,...
1425,臺大公館校區,30,0,25,2024-07-02 20:41:14
1426,臺大公館校區,20,0,20,2024-07-02 20:53:19
1427,臺大公館校區,24,16,7,2024-07-02 20:50:29
1428,臺大公館校區,40,7,33,2024-07-02 20:52:19


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1430 entries, 0 to 1429
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   行政區域    1430 non-null   object        
 1   總數量     1430 non-null   int64         
 2   可借數量    1430 non-null   int64         
 3   可還數量    1430 non-null   int64         
 4   時間      1430 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 56.0+ KB


DataFrame.groupby(by=None, axis=_NoDefault.no_default, level=None, as_index=True, sort=True, group_keys=True, observed=_NoDefault.no_default, dropna=True)[source]

- by=[col1, col2]：數據的分組。
- {col3: func3, col4: func4}：數據的處理。

In [4]:

df1 = df.set_index(keys=["行政區域"])
df1.groupby(level='行政區域')[['總數量','可借數量','可還數量']].sum()
# groupby 是一個可以把數據組合（group）的功能。

,總數量,可借數量,可還數量
行政區域,,,
中山區,3819,1251,2528
中正區,3209,1323,1849
信義區,3331,818,2399
內湖區,3499,1217,2239
北投區,2472,823,1618
南港區,2317,1024,1264
士林區,2962,1110,1763
大同區,1630,526,1092
大安區,4987,1909,2988
